# Validación de Datos: Tablas INE

Este notebook valida la calidad, integridad y coherencia de **TODAS las tablas INE** generadas por el proceso ETL.

## Arquitectura Modular

- **Este módulo**: Validación de tablas INE (Instituto Nacional de Estadística)
- **Módulo hermano**: `02b_validacion_EUROSTAT.ipynb` (tablas EUROSTAT)
- **Módulo de integración**: `02c_validacion_integracion.ipynb` (coherencia entre fuentes)
- **Orquestador**: `02_run_validation.py` (ejecuta todos los módulos)

## Framework de Validación

Usa el framework reutilizable en `utils/validation_framework.py` con reglas declarativas en `utils/validation_rules.py`.

## Salida

- **Reportes JSON/CSV**: `data/validated/logs/`
- **NO crea tablas** en SQL Server (las tablas originales permanecen intactas)

---

In [1]:
# 0. CONFIGURACIÓN Y FRAMEWORK
import sys
import pandas as pd
import numpy as np
import pyodbc
from datetime import datetime

# Añadir utils al path
sys.path.append("../../")

# Importar framework de validación
from utils.validation_framework import (
    ValidationReport,
    check_schema,
    check_uniqueness,
    check_nulls,
    check_conditional_nulls,
    check_range,
    check_time_coherence,
    check_year_continuity,
)
from utils.validation_rules import get_rules, INE_VALIDATION_RULES
from utils.config import DB_CONNECTION_STRING, MAX_NULL_PERCENT

# Conexión a SQL Server
conn = pyodbc.connect(DB_CONNECTION_STRING)

print("=" * 80)
print("MÓDULO DE VALIDACIÓN: TABLAS INE")
print("=" * 80)
print(f"✅ Framework de validación cargado")
print(f"📊 Tablas INE configuradas: {len(INE_VALIDATION_RULES)}")
print(f"🔧 Umbrales: MAX_NULL_PERCENT={MAX_NULL_PERCENT*100}%")
print(f"⏰ Timestamp: {datetime.now().isoformat()}")

MÓDULO DE VALIDACIÓN: TABLAS INE
✅ Framework de validación cargado
📊 Tablas INE configuradas: 13
🔧 Umbrales: MAX_NULL_PERCENT=5.0%
⏰ Timestamp: 2025-11-20T12:15:02.946603


---

## 1. Identificación de Tablas INE

---

In [2]:
# 1. Obtener lista de tablas INE en la base de datos
print("=" * 80)
print("IDENTIFICACIÓN DE TABLAS INE")
print("=" * 80)

cursor = conn.cursor()
cursor.execute(
    """
    SELECT TABLE_NAME 
    FROM INFORMATION_SCHEMA.TABLES 
    WHERE TABLE_TYPE = 'BASE TABLE' 
      AND TABLE_NAME LIKE 'INE_%'
      AND TABLE_NAME NOT LIKE 'VALIDATED_%'
    ORDER BY TABLE_NAME
"""
)

ine_tables = [row[0] for row in cursor.fetchall()]

print(f"\n📊 Tablas INE encontradas: {len(ine_tables)}")
print(f"\n📋 Lista de tablas:")
for i, table in enumerate(ine_tables, 1):
    # Verificar si tiene reglas configuradas
    has_rules = table in INE_VALIDATION_RULES
    status_icon = "✅" if has_rules else "⚠️"
    status_text = "CON REGLAS" if has_rules else "SIN REGLAS"
    print(f"   {i:2d}. {status_icon} {table} ({status_text})")

# Estadísticas
tables_with_rules = sum(1 for t in ine_tables if t in INE_VALIDATION_RULES)
tables_without_rules = len(ine_tables) - tables_with_rules

print(f"\n📈 Resumen:")
print(f"   ✅ Con reglas: {tables_with_rules}")
print(f"   ⚠️  Sin reglas: {tables_without_rules}")

IDENTIFICACIÓN DE TABLAS INE

📊 Tablas INE encontradas: 13

📋 Lista de tablas:
    1. ✅ INE_AROPE_CCAA (CON REGLAS)
    2. ✅ INE_AROPE_Edad_Sexo (CON REGLAS)
    3. ✅ INE_AROPE_Hogar (CON REGLAS)
    4. ✅ INE_AROPE_Laboral (CON REGLAS)
    5. ✅ INE_Carencia_Material_Decil (CON REGLAS)
    6. ✅ INE_Gasto_Medio_Hogar_Quintil (CON REGLAS)
    7. ✅ INE_Gini_S80S20_CCAA (CON REGLAS)
    8. ✅ INE_IPC_Nacional (CON REGLAS)
    9. ✅ INE_IPC_Sectorial_ECOICOP (CON REGLAS)
   10. ✅ INE_Poblacion_Edad_Sexo_CCAA (CON REGLAS)
   11. ✅ INE_Poblacion_Edad_Sexo_Nacionalidad (CON REGLAS)
   12. ✅ INE_Renta_Media_Decil (CON REGLAS)
   13. ✅ INE_Umbral_Pobreza_Hogar (CON REGLAS)

📈 Resumen:
   ✅ Con reglas: 13
   ⚠️  Sin reglas: 0


---

## 2. Función de Validación

---

In [3]:
def validate_ine_table(table_name: str, conn, save_report: bool = True) -> dict:
    """
    Valida una tabla INE usando el framework y reglas configuradas.

    Args:
        table_name: Nombre de la tabla INE a validar
        conn: Conexión a SQL Server
        save_report: Si True, guarda el reporte de validación en archivos JSON/CSV

    Returns:
        Diccionario con resultados de validación
    """
    print("\n" + "=" * 80)
    print(f"VALIDANDO: {table_name}")
    print("=" * 80)

    # Crear reporte
    report = ValidationReport(table_name)

    # Cargar tabla
    try:
        df = pd.read_sql(f"SELECT * FROM {table_name}", conn)
        print(f"✅ Cargada: {len(df)} registros, {len(df.columns)} columnas")
        report.records_original = len(df)
    except Exception as e:
        print(f"❌ Error al cargar tabla: {e}")
        report.add_error(f"Error al cargar tabla: {e}")
        if save_report:
            report.save_json()
            report.save_csv()
        return {
            "table": table_name,
            "status": "ERROR",
            "error": str(e),
            "records_original": 0,
            "records_excluded": 0,
            "errors": 1,
            "warnings": 0,
        }

    # Obtener reglas
    rules = get_rules(table_name)

    if not rules:
        print(f"⚠️ No hay reglas configuradas para {table_name}")
        report.add_warning("No hay reglas de validación configuradas para esta tabla")

        if save_report:
            report.save_json()
            report.save_csv()

        return {
            "table": table_name,
            "status": "NO_RULES",
            "records_original": len(df),
            "records_excluded": 0,
            "errors": 0,
            "warnings": 1,
        }

    print(f"📋 Reglas encontradas: {list(rules.keys())}")

    # 1. Validar esquema
    if "expected_columns" in rules or "expected_types" in rules:
        check_schema(
            df,
            expected_columns=rules.get("expected_columns", []),
            expected_types=rules.get("expected_types", {}),
            report=report,
        )

    # 2. Validar unicidad
    if "primary_key" in rules:
        pk_columns = rules["primary_key"]
        missing_pk_cols = [col for col in pk_columns if col not in df.columns]
        if missing_pk_cols:
            report.add_warning(f"Columnas de PK no encontradas: {missing_pk_cols}")
        else:
            check_uniqueness(df, primary_key=pk_columns, report=report)

    # 3. Validar nulos
    if "critical_columns" in rules:
        critical_cols = [col for col in rules["critical_columns"] if col in df.columns]
        if critical_cols:
            check_nulls(
                df,
                critical_columns=critical_cols,
                max_null_percent=MAX_NULL_PERCENT,
                report=report,
            )

    # 3b. Validar nulos condicionales
    if "conditional_nulls" in rules:
        check_conditional_nulls(df, rules.get("conditional_nulls", {}), report=report)

    # 4. Validar rangos
    if "range_checks" in rules:
        for column, (min_val, max_val) in rules["range_checks"].items():
            if column in df.columns:
                check_range(
                    df, column=column, min_val=min_val, max_val=max_val, report=report
                )

    # 5. Validar continuidad temporal
    if "expected_years" in rules:
        year_column = "Año" if "Año" in df.columns else None
        if year_column:
            check_year_continuity(
                df,
                year_column=year_column,
                expected_years=rules["expected_years"],
                report=report,
            )

    # 6. Contar registros a excluir (sin modificar la BD)
    records_excluded = 0
    if "exclude_categories" in rules:
        for column, categories in rules["exclude_categories"].items():
            if column in df.columns:
                for category in categories:
                    count = len(df[df[column] == category])
                    if count > 0:
                        records_excluded += count
                        report.add_warning(
                            f"Encontrados {count} registros de categoría '{category}' ",
                            f"en columna '{column}' (se recomienda excluir en análisis)",
                        )

    report.records_excluded = records_excluded

    # 7. Guardar reporte
    if save_report:
        report.save_json()
        report.save_csv()

    # Imprimir resumen
    print(f"\n📊 Resumen:")
    print(f"   Registros: {report.records_original}")
    print(f"   Advertencias de exclusión: {report.records_excluded}")
    print(f"   Errores: {len(report.errors)}")
    print(f"   Advertencias: {len(report.warnings)}")
    print(f"   Estado: {'✅ PASSED' if not report.has_errors() else '❌ FAILED'}")

    return {
        "table": table_name,
        "status": "PASSED" if not report.has_errors() else "FAILED",
        "records_original": len(df),
        "records_excluded": records_excluded,
        "errors": len(report.errors),
        "warnings": len(report.warnings),
    }


print("✅ Función validate_ine_table() definida")

✅ Función validate_ine_table() definida


---

## 3. Validación Masiva de Tablas INE

---

In [4]:
# Validar todas las tablas INE
print("=" * 80)
print("VALIDACIÓN MASIVA: TODAS LAS TABLAS INE")
print("=" * 80)
print(f"⏰ Inicio: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"📋 Tablas a validar: {len(ine_tables)}\n")

# Almacenar resultados
results_ine = []

for i, table in enumerate(ine_tables, 1):
    print(f"\n[{i}/{len(ine_tables)}] Procesando: {table}")
    result = validate_ine_table(table, conn, save_report=True)
    results_ine.append(result)

print(f"\n⏰ Fin: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

VALIDACIÓN MASIVA: TODAS LAS TABLAS INE
⏰ Inicio: 2025-11-20 12:15:03
📋 Tablas a validar: 13


[1/13] Procesando: INE_AROPE_CCAA

VALIDANDO: INE_AROPE_CCAA


✅ Cargada: 680 registros, 4 columnas
📋 Reglas encontradas: ['primary_key', 'critical_columns', 'expected_columns', 'expected_types', 'range_checks', 'expected_years', 'coherence_checks']
[REPORT] Report saved: C:\Users\mario\Desktop\Projects\desigualdad_social_etl\notebooks\00_etl\..\..\data\validated\logs\INE_AROPE_CCAA_20251120_121503.json
[REPORT] Report saved: C:\Users\mario\Desktop\Projects\desigualdad_social_etl\notebooks\00_etl\..\..\data\validated\logs\INE_AROPE_CCAA_20251120_121503.csv

📊 Resumen:
   Registros: 680
   Advertencias de exclusión: 0
   Errores: 0
   Advertencias: 2
   Estado: ✅ PASSED

[2/13] Procesando: INE_AROPE_Edad_Sexo

VALIDANDO: INE_AROPE_Edad_Sexo
✅ Cargada: 408 registros, 5 columnas
📋 Reglas encontradas: ['primary_key', 'critical_columns', 'expected_columns', 'expected_types', 'range_checks', 'expected_years']
[REPORT] Report saved: C:\Users\mario\Desktop\Projects\desigualdad_social_etl\notebooks\00_etl\..\..\data\validated\logs\INE_AROPE_Edad_Sexo_20251

✅ Cargada: 308 registros, 4 columnas
📋 Reglas encontradas: ['primary_key', 'critical_columns', 'expected_columns', 'expected_types', 'range_checks', 'expected_years', 'exclude_categories', 'expected_indicators', 'coherence_checks']
[REPORT] Report saved: C:\Users\mario\Desktop\Projects\desigualdad_social_etl\notebooks\00_etl\..\..\data\validated\logs\INE_AROPE_Hogar_20251120_121503.json
[REPORT] Report saved: C:\Users\mario\Desktop\Projects\desigualdad_social_etl\notebooks\00_etl\..\..\data\validated\logs\INE_AROPE_Hogar_20251120_121503.csv

📊 Resumen:
   Registros: 308
   Advertencias de exclusión: 0
   Errores: 0
   Advertencias: 0
   Estado: ✅ PASSED

[4/13] Procesando: INE_AROPE_Laboral

VALIDANDO: INE_AROPE_Laboral
✅ Cargada: 152 registros, 5 columnas
📋 Reglas encontradas: ['primary_key', 'critical_columns', 'expected_columns', 'expected_types', 'range_checks', 'expected_years']
[REPORT] Report saved: C:\Users\mario\Desktop\Projects\desigualdad_social_etl\notebooks\00_etl\..\..\da

✅ Cargada: 1683 registros, 4 columnas
📋 Reglas encontradas: ['primary_key', 'critical_columns', 'range_checks', 'expected_years']
[REPORT] Report saved: C:\Users\mario\Desktop\Projects\desigualdad_social_etl\notebooks\00_etl\..\..\data\validated\logs\INE_Carencia_Material_Decil_20251120_121503.json
[REPORT] Report saved: C:\Users\mario\Desktop\Projects\desigualdad_social_etl\notebooks\00_etl\..\..\data\validated\logs\INE_Carencia_Material_Decil_20251120_121503.csv

📊 Resumen:
   Registros: 1683
   Advertencias de exclusión: 0
   Errores: 0
   Advertencias: 0
   Estado: ✅ PASSED

[6/13] Procesando: INE_Gasto_Medio_Hogar_Quintil

VALIDANDO: INE_Gasto_Medio_Hogar_Quintil
✅ Cargada: 5616 registros, 5 columnas
📋 Reglas encontradas: ['primary_key', 'critical_columns', 'range_checks', 'expected_years']
[REPORT] Report saved: C:\Users\mario\Desktop\Projects\desigualdad_social_etl\notebooks\00_etl\..\..\data\validated\logs\INE_Gasto_Medio_Hogar_Quintil_20251120_121503.json
[REPORT] Report saved

C:\Users\mario\AppData\Local\Temp\ipykernel_26928\722193239.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(f'SELECT * FROM {table_name}', conn)
C:\Users\mario\AppData\Local\Temp\ipykernel_26928\722193239.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(f'SELECT * FROM {table_name}', conn)
C:\Users\mario\AppData\Local\Temp\ipykernel_26928\722193239.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(f'SELECT * FROM {table_name}', conn)
C:\Users\mario\AppData\Local

---

## 4. Resumen Ejecutivo

---

In [5]:
# Resumen consolidado
print("=" * 80)
print("RESUMEN EJECUTIVO: VALIDACIÓN TABLAS INE")
print("=" * 80)

df_results = pd.DataFrame(results_ine)

print(f"\n📊 TABLAS PROCESADAS: {len(results_ine)}")
print("\n" + "=" * 80)
print("DETALLE POR TABLA")
print("=" * 80)
print(df_results.to_string(index=False))

# Estadísticas globales
total_errors = df_results["errors"].sum()
total_warnings = df_results["warnings"].sum()
total_original = df_results["records_original"].sum()
total_excluded = df_results["records_excluded"].sum()

# Contar estados
passed = len(df_results[df_results["status"] == "PASSED"])
failed = len(df_results[df_results["status"] == "FAILED"])
no_rules = len(df_results[df_results["status"] == "NO_RULES"])
errors = len(df_results[df_results["status"] == "ERROR"])

print("\n" + "=" * 80)
print("ESTADÍSTICAS GLOBALES")
print("=" * 80)
print(f"📊 Registros totales: {total_original:,}")
print(
    f"✂️  Registros con advertencia de exclusión: {total_excluded:,} ({total_excluded/total_original*100:.2f}%)"
)
print(f"❌ Total errores: {total_errors}")
print(f"⚠️  Total advertencias: {total_warnings}")

print(f"\n📈 Estado de validación:")
print(f"   ✅ PASSED: {passed}")
print(f"   ❌ FAILED: {failed}")
print(f"   ⚠️  NO_RULES: {no_rules}")
print(f"   🚫 ERROR: {errors}")

if total_errors == 0 and failed == 0:
    print("\n🎉 ¡VALIDACIÓN COMPLETADA SIN ERRORES CRÍTICOS!")
else:
    print(f"\n⚠️  Se encontraron {total_errors} errores en {failed} tablas.")

print("\n" + "=" * 80)
print("REPORTES GENERADOS")
print("=" * 80)
print(f"📁 Ubicación: ../../data/validated/logs/")
print(f"📄 Formato: JSON y CSV por cada tabla")
print(
    f"✅ Total reportes: {len(results_ine) * 2} archivos ({len(results_ine)} JSON + {len(results_ine)} CSV)"
)

print("\n" + "=" * 80)
print("🎯 PRÓXIMOS PASOS")
print("=" * 80)
print("1. Revisar reportes JSON/CSV en data/validated/logs/")
print("2. Corregir errores críticos si los hay (tablas FAILED)")
print("3. Configurar reglas para tablas sin validación (NO_RULES)")
print("4. Ejecutar 02b_validacion_EUROSTAT.ipynb")
print("5. Ejecutar 02c_validacion_integracion.ipynb")
print("=" * 80)

RESUMEN EJECUTIVO: VALIDACIÓN TABLAS INE

📊 TABLAS PROCESADAS: 13

DETALLE POR TABLA
                               table status  records_original  records_excluded  errors  warnings
                      INE_AROPE_CCAA PASSED               680                 0       0         2
                 INE_AROPE_Edad_Sexo PASSED               408                 0       0         0
                     INE_AROPE_Hogar PASSED               308                 0       0         0
                   INE_AROPE_Laboral PASSED               152                 0       0         0
         INE_Carencia_Material_Decil PASSED              1683                 0       0         0
       INE_Gasto_Medio_Hogar_Quintil PASSED              5616                 0       0         0
                INE_Gini_S80S20_CCAA PASSED               340                 0       0         0
                    INE_IPC_Nacional PASSED                24                 0       0         0
           INE_IPC_Sectorial_ECOI

In [6]:
# Cerrar conexión
conn.close()
print("✅ Conexión a SQL Server cerrada")

✅ Conexión a SQL Server cerrada
